# Exploración del Dataset - Cajas VR

Este notebook permite explorar y validar el dataset antes del entrenamiento.

## Objetivos
1. Verificar la estructura del dataset
2. Visualizar imágenes con sus anotaciones
3. Analizar la distribución de clases
4. Detectar posibles problemas en las anotaciones

---

## 1. Setup e Importaciones

In [ ]:
# =============================================================================
# Importaciones necesarias
# =============================================================================

import os
import sys
from pathlib import Path
from collections import Counter

# Visualización
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Procesamiento de imágenes
from PIL import Image
import cv2

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Configurar el directorio del proyecto
PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / 'data'

print(f"Directorio del proyecto: {PROJECT_ROOT}")
print(f"Directorio de datos: {DATA_DIR}")

## 2. Configuración del Dataset

Define las clases y verifica la estructura de carpetas.

In [ ]:
# =============================================================================
# Configuración de clases
# IMPORTANTE: Ajusta estos nombres según tu dataset de Roboflow
# =============================================================================

CLASS_NAMES = {
    0: 'vr_box_type_a',    # Clase 0: Ajusta el nombre
    1: 'vr_box_type_b',    # Clase 1: Ajusta el nombre
}

# Colores para visualización (RGB)
CLASS_COLORS = {
    0: '#FF6B6B',  # Rojo coral
    1: '#4ECDC4',  # Turquesa
}

print("Clases configuradas:")
for idx, name in CLASS_NAMES.items():
    print(f"  {idx}: {name} ({CLASS_COLORS[idx]})")

In [ ]:
# =============================================================================
# Verificar estructura del dataset
# =============================================================================

def check_dataset_structure():
    """Verifica que la estructura del dataset sea correcta."""
    
    splits = ['train', 'valid', 'test']
    structure_ok = True
    
    print("Verificando estructura del dataset...\n")
    
    for split in splits:
        split_dir = DATA_DIR / split
        images_dir = split_dir / 'images'
        labels_dir = split_dir / 'labels'
        
        print(f"📁 {split}/")
        
        # Verificar carpeta de imágenes
        if images_dir.exists():
            num_images = len(list(images_dir.glob('*.[jJpP][pPnN][gG]')))
            print(f"   ├── images/: {num_images} imágenes ✅")
        else:
            print(f"   ├── images/: NO ENCONTRADA ❌")
            structure_ok = False
        
        # Verificar carpeta de labels
        if labels_dir.exists():
            num_labels = len(list(labels_dir.glob('*.txt')))
            print(f"   └── labels/: {num_labels} archivos ✅")
        else:
            print(f"   └── labels/: NO ENCONTRADA ❌")
            structure_ok = False
        
        print()
    
    if structure_ok:
        print("✅ Estructura del dataset correcta")
    else:
        print("❌ Hay problemas con la estructura del dataset")
        print("   Asegúrate de exportar el dataset de Roboflow en formato YOLOv8")
    
    return structure_ok

dataset_ok = check_dataset_structure()

## 3. Análisis de Distribución de Clases

Es importante verificar que las clases estén balanceadas para evitar sesgo en el modelo.

In [ ]:
# =============================================================================
# Funciones de utilidad para parsear anotaciones YOLO
# =============================================================================

def parse_yolo_label(label_path):
    """
    Parsea un archivo de label YOLO.
    
    Formato YOLO: class_id x_center y_center width height (normalizados 0-1)
    
    Args:
        label_path: Ruta al archivo .txt
        
    Returns:
        Lista de diccionarios con las anotaciones
    """
    annotations = []
    
    if not Path(label_path).exists():
        return annotations
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                annotations.append({
                    'class_id': int(parts[0]),
                    'x_center': float(parts[1]),
                    'y_center': float(parts[2]),
                    'width': float(parts[3]),
                    'height': float(parts[4]),
                })
    
    return annotations


def collect_all_annotations(split='train'):
    """
    Recolecta todas las anotaciones de un split.
    
    Args:
        split: 'train', 'valid', o 'test'
        
    Returns:
        DataFrame con todas las anotaciones
    """
    labels_dir = DATA_DIR / split / 'labels'
    images_dir = DATA_DIR / split / 'images'
    
    all_annotations = []
    
    if not labels_dir.exists():
        print(f"⚠️ Directorio no encontrado: {labels_dir}")
        return pd.DataFrame()
    
    for label_file in labels_dir.glob('*.txt'):
        annotations = parse_yolo_label(label_file)
        
        # Buscar imagen correspondiente
        img_name = label_file.stem
        img_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
            potential_path = images_dir / f"{img_name}{ext}"
            if potential_path.exists():
                img_path = potential_path
                break
        
        for ann in annotations:
            ann['split'] = split
            ann['label_file'] = label_file.name
            ann['image_file'] = img_path.name if img_path else None
            all_annotations.append(ann)
    
    return pd.DataFrame(all_annotations)

print("Funciones de utilidad cargadas ✅")

In [ ]:
# =============================================================================
# Recolectar y analizar anotaciones de todos los splits
# =============================================================================

# Recolectar anotaciones
df_train = collect_all_annotations('train')
df_valid = collect_all_annotations('valid')
df_test = collect_all_annotations('test')

# Combinar todos los datos
df_all = pd.concat([df_train, df_valid, df_test], ignore_index=True)

if len(df_all) > 0:
    print(f"\n📊 Resumen del Dataset:")
    print(f"   Total de anotaciones: {len(df_all)}")
    print(f"   - Train: {len(df_train)}")
    print(f"   - Valid: {len(df_valid)}")
    print(f"   - Test: {len(df_test)}")
else:
    print("⚠️ No se encontraron anotaciones. Asegúrate de que el dataset esté en data/")

In [ ]:
# =============================================================================
# Visualizar distribución de clases
# =============================================================================

if len(df_all) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico 1: Distribución total de clases
    class_counts = df_all['class_id'].value_counts().sort_index()
    colors = [CLASS_COLORS.get(i, '#666666') for i in class_counts.index]
    
    ax1 = axes[0]
    bars = ax1.bar(class_counts.index, class_counts.values, color=colors, edgecolor='black')
    ax1.set_xlabel('Clase')
    ax1.set_ylabel('Número de instancias')
    ax1.set_title('Distribución de Clases (Total)')
    ax1.set_xticks(list(CLASS_NAMES.keys()))
    ax1.set_xticklabels([CLASS_NAMES.get(i, f'Clase {i}') for i in class_counts.index])
    
    # Añadir valores sobre las barras
    for bar, count in zip(bars, class_counts.values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                 str(count), ha='center', va='bottom', fontweight='bold')
    
    # Gráfico 2: Distribución por split
    ax2 = axes[1]
    splits = ['train', 'valid', 'test']
    x = np.arange(len(CLASS_NAMES))
    width = 0.25
    
    for i, split in enumerate(splits):
        df_split = df_all[df_all['split'] == split]
        counts = [len(df_split[df_split['class_id'] == c]) for c in CLASS_NAMES.keys()]
        ax2.bar(x + i*width, counts, width, label=split.capitalize())
    
    ax2.set_xlabel('Clase')
    ax2.set_ylabel('Número de instancias')
    ax2.set_title('Distribución de Clases por Split')
    ax2.set_xticks(x + width)
    ax2.set_xticklabels([CLASS_NAMES.get(i, f'Clase {i}') for i in CLASS_NAMES.keys()])
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Calcular balance de clases
    print("\n📈 Balance de clases:")
    total = len(df_all)
    for class_id in CLASS_NAMES.keys():
        count = len(df_all[df_all['class_id'] == class_id])
        percentage = (count / total) * 100
        print(f"   {CLASS_NAMES[class_id]}: {count} ({percentage:.1f}%)")
    
    # Advertencia si hay desbalance
    percentages = [(len(df_all[df_all['class_id'] == c]) / total) * 100 for c in CLASS_NAMES.keys()]
    if max(percentages) / min(percentages) > 3:
        print("\n⚠️ ADVERTENCIA: Las clases están muy desbalanceadas")
        print("   Considera agregar más datos de la clase minoritaria")
else:
    print("No hay datos para visualizar")

## 4. Análisis de Tamaños de Bounding Boxes

Entender el tamaño de los objetos ayuda a configurar mejor el modelo.

In [ ]:
# =============================================================================
# Analizar tamaños de bounding boxes
# =============================================================================

if len(df_all) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Gráfico 1: Distribución de anchos
    ax1 = axes[0]
    for class_id in CLASS_NAMES.keys():
        df_class = df_all[df_all['class_id'] == class_id]
        ax1.hist(df_class['width'], bins=20, alpha=0.6, 
                 label=CLASS_NAMES[class_id], color=CLASS_COLORS[class_id])
    ax1.set_xlabel('Ancho (normalizado)')
    ax1.set_ylabel('Frecuencia')
    ax1.set_title('Distribución de Anchos')
    ax1.legend()
    
    # Gráfico 2: Distribución de altos
    ax2 = axes[1]
    for class_id in CLASS_NAMES.keys():
        df_class = df_all[df_all['class_id'] == class_id]
        ax2.hist(df_class['height'], bins=20, alpha=0.6, 
                 label=CLASS_NAMES[class_id], color=CLASS_COLORS[class_id])
    ax2.set_xlabel('Alto (normalizado)')
    ax2.set_ylabel('Frecuencia')
    ax2.set_title('Distribución de Altos')
    ax2.legend()
    
    # Gráfico 3: Scatter de ancho vs alto
    ax3 = axes[2]
    for class_id in CLASS_NAMES.keys():
        df_class = df_all[df_all['class_id'] == class_id]
        ax3.scatter(df_class['width'], df_class['height'], alpha=0.5,
                   label=CLASS_NAMES[class_id], color=CLASS_COLORS[class_id])
    ax3.set_xlabel('Ancho (normalizado)')
    ax3.set_ylabel('Alto (normalizado)')
    ax3.set_title('Ancho vs Alto de Bounding Boxes')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Estadísticas
    print("\n📐 Estadísticas de tamaño de bounding boxes:")
    for class_id in CLASS_NAMES.keys():
        df_class = df_all[df_all['class_id'] == class_id]
        if len(df_class) > 0:
            print(f"\n   {CLASS_NAMES[class_id]}:")
            print(f"      Ancho: {df_class['width'].mean():.3f} ± {df_class['width'].std():.3f}")
            print(f"      Alto:  {df_class['height'].mean():.3f} ± {df_class['height'].std():.3f}")
            area = df_class['width'] * df_class['height']
            print(f"      Área:  {area.mean():.4f} ± {area.std():.4f}")
else:
    print("No hay datos para analizar")

## 5. Visualización de Imágenes con Anotaciones

Verificar visualmente que las anotaciones son correctas.

In [ ]:
# =============================================================================
# Función para visualizar imagen con anotaciones
# =============================================================================

def visualize_image_with_annotations(image_path, label_path, figsize=(12, 8)):
    """
    Muestra una imagen con sus bounding boxes anotados.
    
    Args:
        image_path: Ruta a la imagen
        label_path: Ruta al archivo de label
        figsize: Tamaño de la figura
    """
    # Cargar imagen
    img = Image.open(image_path)
    img_width, img_height = img.size
    
    # Parsear anotaciones
    annotations = parse_yolo_label(label_path)
    
    # Crear figura
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.imshow(img)
    
    # Dibujar cada bounding box
    for ann in annotations:
        # Convertir de formato YOLO (centro, tamaño normalizado) a píxeles
        x_center = ann['x_center'] * img_width
        y_center = ann['y_center'] * img_height
        width = ann['width'] * img_width
        height = ann['height'] * img_height
        
        # Calcular esquina superior izquierda
        x1 = x_center - width / 2
        y1 = y_center - height / 2
        
        # Obtener color y nombre de clase
        class_id = ann['class_id']
        color = CLASS_COLORS.get(class_id, '#666666')
        class_name = CLASS_NAMES.get(class_id, f'Clase {class_id}')
        
        # Dibujar rectángulo
        rect = patches.Rectangle(
            (x1, y1), width, height,
            linewidth=2, edgecolor=color, facecolor='none'
        )
        ax.add_patch(rect)
        
        # Añadir etiqueta
        ax.text(x1, y1 - 5, class_name, color='white', fontsize=10,
               fontweight='bold', bbox=dict(boxstyle='round', facecolor=color, alpha=0.8))
    
    ax.set_title(f"{Path(image_path).name} - {len(annotations)} anotaciones")
    ax.axis('off')
    plt.tight_layout()
    plt.show()

print("Función de visualización cargada ✅")

In [ ]:
# =============================================================================
# Visualizar algunas imágenes de ejemplo
# =============================================================================

def show_random_samples(split='train', n_samples=4):
    """
    Muestra n_samples imágenes aleatorias del split especificado.
    """
    images_dir = DATA_DIR / split / 'images'
    labels_dir = DATA_DIR / split / 'labels'
    
    if not images_dir.exists():
        print(f"⚠️ Directorio no encontrado: {images_dir}")
        return
    
    # Obtener lista de imágenes
    image_files = list(images_dir.glob('*.[jJpP][pPnN][gG]'))
    
    if len(image_files) == 0:
        print(f"⚠️ No se encontraron imágenes en {images_dir}")
        return
    
    # Seleccionar muestras aleatorias
    n_samples = min(n_samples, len(image_files))
    selected = np.random.choice(image_files, n_samples, replace=False)
    
    print(f"\n🖼️ Mostrando {n_samples} imágenes de {split}:\n")
    
    for img_path in selected:
        label_path = labels_dir / f"{img_path.stem}.txt"
        visualize_image_with_annotations(img_path, label_path)

# Mostrar ejemplos del set de entrenamiento
show_random_samples('train', n_samples=4)

## 6. Verificación de Calidad de Datos

Detectar posibles problemas en el dataset.

In [ ]:
# =============================================================================
# Verificar calidad de datos
# =============================================================================

def check_data_quality(split='train'):
    """
    Verifica la calidad de los datos de un split.
    
    Busca:
    - Imágenes sin labels
    - Labels sin imágenes
    - Bounding boxes fuera de rango
    - Clases inválidas
    """
    images_dir = DATA_DIR / split / 'images'
    labels_dir = DATA_DIR / split / 'labels'
    
    issues = []
    
    if not images_dir.exists() or not labels_dir.exists():
        print(f"⚠️ Directorios no encontrados para {split}")
        return
    
    # Obtener sets de archivos
    image_stems = {f.stem for f in images_dir.glob('*.[jJpP][pPnN][gG]')}
    label_stems = {f.stem for f in labels_dir.glob('*.txt')}
    
    # 1. Imágenes sin labels
    missing_labels = image_stems - label_stems
    if missing_labels:
        issues.append(f"⚠️ {len(missing_labels)} imágenes sin archivo de label")
        for name in list(missing_labels)[:5]:
            issues.append(f"   - {name}")
        if len(missing_labels) > 5:
            issues.append(f"   ... y {len(missing_labels) - 5} más")
    
    # 2. Labels sin imágenes
    missing_images = label_stems - image_stems
    if missing_images:
        issues.append(f"⚠️ {len(missing_images)} labels sin imagen correspondiente")
    
    # 3. Verificar contenido de labels
    invalid_bbox = 0
    invalid_class = 0
    empty_labels = 0
    
    for label_file in labels_dir.glob('*.txt'):
        annotations = parse_yolo_label(label_file)
        
        if len(annotations) == 0:
            empty_labels += 1
            continue
        
        for ann in annotations:
            # Verificar rango de coordenadas
            if not (0 <= ann['x_center'] <= 1 and 0 <= ann['y_center'] <= 1):
                invalid_bbox += 1
            if not (0 < ann['width'] <= 1 and 0 < ann['height'] <= 1):
                invalid_bbox += 1
            
            # Verificar clase válida
            if ann['class_id'] not in CLASS_NAMES:
                invalid_class += 1
    
    if empty_labels > 0:
        issues.append(f"ℹ️ {empty_labels} imágenes sin objetos (labels vacíos)")
    
    if invalid_bbox > 0:
        issues.append(f"❌ {invalid_bbox} bounding boxes con coordenadas inválidas")
    
    if invalid_class > 0:
        issues.append(f"❌ {invalid_class} anotaciones con clase inválida")
    
    # Mostrar resultados
    print(f"\n🔍 Verificación de calidad - {split.upper()}")
    print("=" * 50)
    
    if not issues:
        print("✅ No se encontraron problemas")
    else:
        for issue in issues:
            print(issue)
    
    print(f"\n   Total imágenes: {len(image_stems)}")
    print(f"   Total labels: {len(label_stems)}")

# Verificar cada split
for split in ['train', 'valid', 'test']:
    check_data_quality(split)

## 7. Resumen y Recomendaciones

Basado en el análisis, aquí están las recomendaciones para el entrenamiento.

In [ ]:
# =============================================================================
# Generar resumen y recomendaciones
# =============================================================================

print("\n" + "=" * 60)
print("📋 RESUMEN DEL DATASET")
print("=" * 60)

if len(df_all) > 0:
    print(f"\n📊 Estadísticas generales:")
    print(f"   - Total de anotaciones: {len(df_all)}")
    print(f"   - Clases: {len(CLASS_NAMES)}")
    
    print(f"\n📁 Distribución por split:")
    for split in ['train', 'valid', 'test']:
        count = len(df_all[df_all['split'] == split])
        print(f"   - {split}: {count} anotaciones")
    
    # Recomendaciones basadas en los datos
    print(f"\n💡 RECOMENDACIONES:")
    
    # 1. Tamaño del dataset
    train_count = len(df_train)
    if train_count < 100:
        print(f"   ⚠️ Dataset pequeño ({train_count} anotaciones). Considera:")
        print(f"      - Usar augmentation agresivo")
        print(f"      - Reducir épocas (50-100) para evitar overfitting")
        print(f"      - Agregar más datos si es posible")
    elif train_count < 500:
        print(f"   ℹ️ Dataset moderado ({train_count} anotaciones).")
        print(f"      - Augmentation estándar debería funcionar bien")
        print(f"      - 100-200 épocas recomendadas")
    else:
        print(f"   ✅ Dataset de buen tamaño ({train_count} anotaciones).")
        print(f"      - Puedes reducir augmentation si es necesario")
        print(f"      - 100-300 épocas según resultados")
    
    # 2. Balance de clases
    total = len(df_all)
    percentages = {c: (len(df_all[df_all['class_id'] == c]) / total) * 100 for c in CLASS_NAMES.keys()}
    max_pct = max(percentages.values())
    min_pct = min(percentages.values())
    
    if max_pct / min_pct > 3:
        print(f"\n   ⚠️ Clases desbalanceadas (ratio {max_pct/min_pct:.1f}:1)")
        print(f"      - Considera usar class weights")
        print(f"      - O agregar más datos de la clase minoritaria")
    else:
        print(f"\n   ✅ Clases relativamente balanceadas")
    
    # 3. Tamaño de objetos
    mean_area = (df_all['width'] * df_all['height']).mean()
    if mean_area < 0.01:
        print(f"\n   ℹ️ Objetos pequeños (área promedio: {mean_area:.4f})")
        print(f"      - Considera usar imgsz=1280 para mejor detección")
        print(f"      - Pero requiere más VRAM")
    elif mean_area > 0.1:
        print(f"\n   ✅ Objetos de tamaño mediano-grande (área: {mean_area:.4f})")
        print(f"      - imgsz=640 debería funcionar bien")
    
    print("\n" + "=" * 60)
    print("Listo para entrenar: python scripts/train.py")
    print("=" * 60)
    
else:
    print("\n⚠️ No hay datos en el dataset")
    print("   Sigue las instrucciones en README.md para:")
    print("   1. Capturar screenshots VR")
    print("   2. Anotar con Roboflow")
    print("   3. Exportar en formato YOLOv8")
    print("   4. Colocar los archivos en data/")

---

## Próximos Pasos

1. Si el dataset se ve bien, procede al entrenamiento:
   ```bash
   python scripts/train.py
   ```

2. Si encontraste problemas, corrígelos en Roboflow y re-exporta.

3. Después del entrenamiento, documenta los resultados en `docs/benchmarks.md`